In [12]:
%load_ext autoreload
%autoreload 2    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setup

In [13]:
DB_NAME = "./storm.db"
# DB_NAME = "./hipporag.db"
DB_NAME = "./gpt_review.db"

In [14]:
from broai.prompt_management.core import PromptGenerator
from broai.prompt_management.interface import Persona, Instructions, Examples, Example
from pydantic import BaseModel, Field
from typing import List
from broai.experiments.bro_agent import BroAgent
import json
from broai.interface import Context, Contexts
from broai.experiments.vector_store import DuckVectorStore

In [15]:
from package.jargon_store import JargonStore, JargonRecord

In [16]:
from broai.experiments.cross_encoder import ReRanker
rr = ReRanker()

/tmp/ipykernel_15363/2186289470.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [17]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/tmp/ipykernel_15363/2466456318.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 212549.19it/s]


In [7]:
raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)
jargon_memory = JargonStore(db_name=DB_NAME, table="jargon_memory")

/tmp/ipykernel_15363/1440006260.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_15363/1440006260.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
/tmp/ipykernel_15363/1440006260.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)


# Agent Flows: 
- JargonDetector
- JargonEditor

In [18]:
from package.online_flow import OnlineFlow, JargonFlow, KnowledgeFlow, Payload
from agents.jargon_detector import JargonDetector
from agents.jargon_editor import JargonEditor
from agents.query_decomposer import QueryDecomposer
from agents.ai_oracle import AIOracle
from agents.kb_oracle import KBOracle

In [36]:
DB_NAME = "./storm.db"

jargon_flow = JargonFlow(
    jargon_memory=JargonStore(db_name=DB_NAME, table="jargon_memory"), 
    jargon_detector=JargonDetector(), 
    jargon_editor=JargonEditor()
)
knowledge_flow = KnowledgeFlow(
    longterm_memory=DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em), 
    reranker=rr, 
    search_method="vector"
)
olf = OnlineFlow(
    oracle=KBOracle(), 
    knowledge_flow=knowledge_flow,
    jargon_flow=jargon_flow,
    query_decomposer=QueryDecomposer(),
)

/tmp/ipykernel_15363/1386738580.py:9: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory=DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em),


In [37]:
%%time

# olf.run(message="What is the dataset used in the study?")
# olf.run(message="What does STORM do in the study?")
olf.run(message="What does STORM stand for?")
# olf.run(message="What does STORM stand for and how does it work? Explain it in step-by-step manner.")
# olf.run(message="What does STORM stand for and how does it work? Could you please explain it like I'm a five years old? I'm really new to this thing.")

print(olf.payload.answer)

STORM stands for Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking.
CPU times: user 660 ms, sys: 32.9 ms, total: 693 ms
Wall time: 4.66 s


In [21]:
DB_NAME = "./hipporag.db"

jargon_flow = JargonFlow(
    jargon_memory=JargonStore(db_name=DB_NAME, table="jargon_memory"), 
    jargon_detector=JargonDetector(), 
    jargon_editor=JargonEditor()
)
knowledge_flow = KnowledgeFlow(
    longterm_memory=DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em), 
    reranker=rr, 
    search_method="vector"
)
olf = OnlineFlow(
    oracle=KBOracle(), 
    knowledge_flow=knowledge_flow,
    jargon_flow=jargon_flow,
    query_decomposer=QueryDecomposer(),
)

/tmp/ipykernel_15363/2648505056.py:9: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory=DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em),


In [35]:
%%time

olf.run(message="What does HIPPORAG stand for?")
# olf.run(message="What does HippoRAG do in the study?")
# olf.run(message="What does HippoRAG stand for and how does it work? Explain it in step-by-step manner.")
# olf.run(message="What does HippoRAG stand for and how does it work? Could you please explain it like I'm a five years old? I'm really new to this thing.")

print(olf.payload.answer)

HippoRAG stands for a novel retrieval framework inspired by human long-term memory, specifically the hippocampal indexing theory of human long-term memory.
CPU times: user 667 ms, sys: 26.9 ms, total: 694 ms
Wall time: 5.07 s


In [38]:
DB_NAME = "./gpt_review.db"

jargon_flow = JargonFlow(
    jargon_memory=JargonStore(db_name=DB_NAME, table="jargon_memory"), 
    jargon_detector=JargonDetector(), 
    jargon_editor=JargonEditor()
)
knowledge_flow = KnowledgeFlow(
    longterm_memory=DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em), 
    reranker=rr, 
    search_method="vector"
)
olf = OnlineFlow(
    oracle=KBOracle(), 
    knowledge_flow=knowledge_flow,
    jargon_flow=jargon_flow,
    query_decomposer=QueryDecomposer(),
)

/tmp/ipykernel_15363/2816527970.py:9: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory=DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em),


In [47]:
%%time

# olf.run(message="What does GPT stand for?")
olf.run(message="What does GPT do in the review?")

print(olf.payload.answer)

According to the review, GPT, or Generative Pre-trained Transformer, is a type of language model that uses deep learning techniques to generate human-like text based on input. It is a pre-trained language model that can perform a wide range of language-related tasks, including text generation, language translation, and text classification.

The review highlights the evolution of GPT models, starting from GPT-1, which was a 12-layer decoder architecture, to GPT-4, which is a more advanced model with 175 billion parameters. The review also discusses the applications of GPT models, including education, healthcare, customer service, and game design.

In the context of education, GPT models can assist in identifying leisure activities, job searching, and lifestyle improvement with personalized recommendations. They can also aid in research writing, automated assessments, and fostering creativity.

In game design, GPT models can be used for content creation, analyzing player abilities, and g